In [92]:
!pip install transformers -q

In [93]:
from transformers import pipeline


In [94]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly id

1) negative 0.7236
2) neutral 0.2287
3) positive 0.0477


In [95]:
#smple input
text = "You are the normal"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
#run our model in the text
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)



# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores



ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 3)}")

1) neutral 0.679
2) positive 0.258
3) negative 0.063


#Making Function for the model input and that return a output

In [107]:
def final_sentiment_scores(comments_reviews):
  text = preprocess(comments_reviews)
  encoded_input = tokenizer(text, return_tensors='pt')
  #run our model in the text
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)



# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores

  ranking = np.argsort(scores)
  scores_dict = {
      'Negative': scores[ranking[0]],
      'Neutral': scores[ranking[1]],
      'Positive': scores[ranking[2]]
  }
  return scores_dict


In [108]:
#Checking examples with my created function
Comment = "you are a good boy"
final_sentiment_scores(Comment)

{'Negative': 0.015491988, 'Neutral': 0.05047896, 'Positive': 0.93402904}

#Now apply model into a csv files

In [109]:
import pandas as pd
import numpy as np

In [110]:
#Read in csv
df = pd.read_csv("/content/reviews.csv")
df.head(10)

,HT-1000,100000000,Three start. I have some issues like some keys don't work on first instance..and is somewhat slow also
0,HT-1001,100000001,Heating and not wake up. Check twice when buy ...
1,HT-1002,100000002,2nd hand laptops!!! The laptop I received alre...
2,HT-1003,100000003,Review: BEST LAPTOP IN A PRICE SEGMENT OF Rs 5...
3,HT-1010,100000004,Review: second hand product. don't buy this. I...
4,HT-1020,100000005,Review: very bad. My product is not working wh...
5,HT-1030,100000006,Review: waste of money. Battery is draining ou...
6,HT-1007,100000007,Review: Very bad product. The Serial number on...
7,HT-1040,100000008,Review: One star. Trackpad not working... trie...
8,HT-1050,100000009,Review: two star.Returned looking for Mac pro.
9,HT-1000,100000010,Review: Three star. very low memory. Not good ...


In [111]:
#Check columns
df.columns

Index(['HT-1000', '100000000',
       'Three start. I have some issues like some keys don't work on first instance..and is somewhat slow also'],
      dtype='object')

In [112]:
#Rename column names
df.columns = ["company", "Comment_id", "Comment_text"]
df.head()

,company,Comment_id,Comment_text
0,HT-1001,100000001,Heating and not wake up. Check twice when buy ...
1,HT-1002,100000002,2nd hand laptops!!! The laptop I received alre...
2,HT-1003,100000003,Review: BEST LAPTOP IN A PRICE SEGMENT OF Rs 5...
3,HT-1010,100000004,Review: second hand product. don't buy this. I...
4,HT-1020,100000005,Review: very bad. My product is not working wh...


In [117]:
#run my function with Comment_text column
from tqdm.notebook import tqdm
res = {}


for i, row in tqdm (df.iterrows(), total = len(df)):
  try:
    myid = row['Comment_id']
    text  = row['Comment_text']
    res[myid] = final_sentiment_scores(text)
  except RuntimeError:
    print(f"Broke for id - {myid}")

  0%|          | 0/84 [00:00<?, ?it/s]

In [ ]:
#Showing results as dictionary
res

#Now add the results to the dataset as columns

In [118]:
#Make a dataframe with my dictionary
df_res = pd.DataFrame(res).T
df_res.head()

,Negative,Neutral,Positive
100000001,0.003048,0.007623,0.989329
100000002,0.003940,0.014547,0.981513
100000003,0.002547,0.012033,0.985420
100000004,0.006175,0.018436,0.975389
100000005,0.003246,0.017117,0.979637


In [134]:
#Add to my previous original csv file
df_res = pd.DataFrame(res).T
df_res = df_res.reset_index().rename(columns = {"index": "Comment_id"})
df_res = df_res.merge(df, how = "right")
df_res.head()

,Comment_id,Negative,Neutral,Positive,company,Comment_text
0,100000001,0.003048,0.007623,0.989329,HT-1001,Heating and not wake up. Check twice when buy ...
1,100000002,0.003940,0.014547,0.981513,HT-1002,2nd hand laptops!!! The laptop I received alre...
2,100000003,0.002547,0.012033,0.985420,HT-1003,Review: BEST LAPTOP IN A PRICE SEGMENT OF Rs 5...
3,100000004,0.006175,0.018436,0.975389,HT-1010,Review: second hand product. don't buy this. I...
4,100000005,0.003246,0.017117,0.979637,HT-1020,Review: very bad. My product is not working wh...
